# TASK 1

Import libraries for all three tasks.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle json files

import re #library for regular expression operations to search strings

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Scrape the wikipedia page.

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
    'User-Agent': 'Mozilla/5.0'
}
page = requests.post(wiki, headers=header)
soup = BeautifulSoup(page.content)

Focus on the content of tables.

In [6]:
tables = soup.find("table")
tables = tables.findAll('p')

Write the information of the wikipedia table to the dataframe.

In [33]:
columns = ["PostalCode","Borough","Neighborhood"]
df = pd.DataFrame(columns = columns)
df

find_borough = re.compile(r'[>](.*?)[<]', re.S)
find_neighborhood = re.compile(r'[(](.*?)[)]', re.S)

x = 0
b = [[],[]] #Store PostalCode, Borough, and Neighborhood Data
for i,rows in enumerate(tables):
    
    #Get PostalCode and Borough
    a = re.findall(p1,str(tables[i]))
    k=0
    for j in range(len(a)):
        if a[j] != '':
            b[k].append(a[j])
            k=k+1
        if k == 2: break
    if a[j] == 'Not assigned': continue
    PostalCode, Borough = b[0][i], b[1][i]
    
    #Get Neighbourhood (between the paranthesis)
    try:
        Neighborhood = re.findall(p2,tables[i].text)[0].replace(' /',', ')
    except:
        #print(i,rows.text)
        Neighborhood = "Queen's Park, Ontario Provincial Government"
        Borough = "Downtown Toronto"
        #No neighbourhood assigned within paranthesis
        #By running the code, the problem is found only at M7A (Queen's Park / Ontario Provincial Government).
        #The problem is actually no borough is assigned. 
        #According to the wikipedia page of Queen's Park, it should be at Downtown Toronto
        #This entry is manually modified here
    
    df.loc[x]=[PostalCode, Borough, Neighborhood]
    x=x+1
   
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [37]:
df.shape

(103, 3)

# TASK 2

Get the geospatial data from the csv file.

In [12]:
df_geospatial = pd.read_csv(r'http://cocl.us/Geospatial_data')
df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join the geospatial data to the task 1 dataframe df.

In [117]:
df2 = df.join(df_geospatial.set_index('Postal Code'), on='PostalCode')
df2.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
